## Simulación de factura

**Método rápido:**

* Path al CSV de consumos horarios descargado
* El peaje contratado (GEN / NOC / VHC, para 2.0A / 2.0DHA / 2.0DHS)
* La potencia contratada en kW
* La zona de impuestos (IVA / IGIC / IPSI)


In [1]:
from pathlib import Path
from pvpcbill import create_bill

# Path al CSV de consumos horarios
p_ejemplos = Path("/Volumes/WORK/PYTHON/pvpcbill/tests/ejemplos_consumo")
p_consumo = p_ejemplos / "consumo_facturado18_02_2020-18_03_2020-R.csv"

assert p_consumo.exists(), "Fix the path to the CSV!!"
print(p_consumo.name)

# Creación directa de factura
factura = await create_bill(
    path_csv_consumo=p_consumo,
    potencia_contratada=4.6,
    tipo_peaje="NOC",
    zona_impuestos="IVA",
)

# La representación del objeto como texto simula el desglose de la factura eléctrica
factura

Download of 720 prices from 2020-02-17 23:00:00+00:00 to 2020-03-18 22:00:00+00:00 in 0.17 sec


consumo_facturado18_02_2020-18_03_2020-R.csv


FACTURA ELÉCTRICA:
--------------------------------------------------------------------------------
* CUPS             	        ES0012345678901234SN
* Fecha inicio             	17/02/2020
* Fecha final              	18/03/2020
* Peaje de acceso          	2.0DHA (Nocturna)
* Potencia contratada      	4.60 kW
* Consumo periodo          	472.93 kWh
* ¿Bono Social?            	No
* Equipo de medida         	0.80 €
* Impuestos                	Península y Baleares (IVA)
* Días facturables         	30
--------------------------------------------------------------------------------

- CÁLCULO DEL TÉRMINO FIJO POR POTENCIA CONTRATADA:
  Peaje acceso potencia:
   4.60 kW x 0.103944 €/kW/día x 30 días (366/2020) = 14.34 €
  Comercialización:
   4.60 kW x 0.008505 €/kW/día x 30 días (366/2020) = 1.17 €
  ==> Término fijo                                                     15.51 €

- CÁLCULO DEL TÉRMINO VARIABLE POR ENERGÍA CONSUMIDA (TARIFA 2.0DHA):
    Periodo 1: 0.111867 €/kWh                   

In [2]:
# Reparto de costes en la factura
p_imp = (
    + factura.data.termino_impuesto_electrico
    + factura.data.termino_equipo_medida
    + factura.data.termino_iva_total
) / factura.data.total
p_ener = factura.data.termino_variable_total / factura.data.total
p_pot = factura.data.termino_fijo_total / factura.data.total
assert abs(p_pot + p_ener + p_imp - 1) < 0.001

print(
    f"El coste de la factura se reparte en:\n  "
    f"* un {100*p_ener:.1f} % por energía consumida,\n  "
    f"* un {100*p_pot:.1f} % por potencia contratada,\n  "
    f"* un {100*p_imp:.1f} % por impuestos aplicados\n\n"
)

print(factura.data.to_json())

El coste de la factura se reparte en:
  * un 52.6 % por energía consumida,
  * un 24.8 % por potencia contratada,
  * un 22.6 % por impuestos aplicados


{
  "config": {
    "tipo_peaje": "NOC",
    "potencia_contratada": 4.6,
    "con_bono_social": false,
    "zona_impuestos": "IVA",
    "alquiler_anual": 9.72,
    "impuesto_electrico": 0.0511269632,
    "cups": "ES0012345678901234SN"
  },
  "num_dias_factura": 30,
  "start": "2020-02-17 00:00:00",
  "end": "2020-03-18 00:00:00",
  "periodos_fact": [
    {
      "billed_days": 30,
      "year": 2020,
      "termino_fijo_peaje_acceso": 14.34,
      "termino_fijo_comercializacion": 1.17,
      "termino_fijo_total": 15.51,
      "energy_periods": [
        {
          "name": "P1",
          "coste_peaje_acceso_tea": 10.544458468,
          "coste_energia_tcu": 8.477372039999999,
          "energia_total": 170.03900000000002
        },
        {
          "name": "P2",
          "coste_peaje_acceso_tea": 0.67090578,
          "coste_ener